<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/trial0_DDI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

## CONFRONTO TRAINING SET E TEST SET

In [ ]:
dev = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/train.tsv', sep='\t')

In [ ]:
print(dev.shape)
print(test.shape)
print(train.shape)

(7244, 3)
(5761, 3)
(18779, 3)


In [ ]:
merge_test = pd.merge(left=train,
                      right=test,
                      on='sentence')

merge_dev = pd.merge(left=train,
                     right=dev,
                     on='sentence')

In [ ]:
print(f'OVERLAPPING DATA: {len(merge_test.loc[merge_test["label_x"] == merge_test["label_y"], :])}')
print(f'OVERLAPPING DATA: {len(merge_dev.loc[merge_dev["label_x"] == merge_dev["label_y"], :])}')

OVERLAPPING DATA: 57
OVERLAPPING DATA: 322


Si osserva che le proporzioni sono costanti circa

In [ ]:
print(dev.label.value_counts())
print(test.label.value_counts())
print(train.label.value_counts())

DDI-false        6240
DDI-effect        396
DDI-mechanism     373
DDI-advise        193
DDI-int            42
Name: label, dtype: int64
DDI-false        4782
DDI-effect        360
DDI-mechanism     302
DDI-advise        221
DDI-int            96
Name: label, dtype: int64
DDI-false        15842
DDI-effect        1212
DDI-mechanism      946
DDI-advise         633
DDI-int            146
Name: label, dtype: int64


## OSSERVAZIONI SUL DATASET

In [3]:
train = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/train.tsv', sep='\t')
dev = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/test.tsv', sep='\t')

In [ ]:
train.label.unique()

array(['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise',
       'DDI-int'], dtype=object)

In [ ]:
print(f'TRAIN:\n{train.label.value_counts()}')
print(f'TEST:\n{test.label.value_counts()}')

TRAIN:
DDI-false        15842
DDI-effect        1212
DDI-mechanism      946
DDI-advise         633
DDI-int            146
Name: label, dtype: int64
TEST:
DDI-false        4782
DDI-effect        360
DDI-mechanism     302
DDI-advise        221
DDI-int            96
Name: label, dtype: int64


In [ ]:
train

,index,sentence,label
0,DDI-DrugBank.d0.s0.p0,The concomitant intake of @DRUG$ and @DRUG$ does not affect the pharmacokinetics of either alcohol or acamprosate.,DDI-false
1,DDI-DrugBank.d0.s0.p1,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or acamprosate.,DDI-false
2,DDI-DrugBank.d0.s0.p2,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either alcohol or @DRUG$.,DDI-false
3,DDI-DrugBank.d0.s0.p3,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either @DRUG$ or acamprosate.,DDI-false
4,DDI-DrugBank.d0.s0.p4,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either alcohol or @DRUG$.,DDI-false
...,...,...,...
18774,DDI-MedLine.d99.s11.p2,"The clearance by means of @DRUG$ and carboxytolbutamide was significantly reduced in both groups (ie, from 901 mL/h to 318 mL/h in the group that received 150 mg of tolbutamide per day and from 723 mL/h to 457 mL/h in the group that received 75 mg of @DRUG$ per day).",DDI-false
18775,DDI-MedLine.d99.s11.p3,"The clearance by means of 4-hydroxytolbutamide and @DRUG$ was significantly reduced in both groups (ie, from 901 mL/h to 318 mL/h in the group that received 150 mg of @DRUG$ per day and from 723 mL/h to 457 mL/h in the group that received 75 mg of tolbutamide per day).",DDI-false
18776,DDI-MedLine.d99.s11.p4,"The clearance by means of 4-hydroxytolbutamide and @DRUG$ was significantly reduced in both groups (ie, from 901 mL/h to 318 mL/h in the group that received 150 mg of tolbutamide per day and from 723 mL/h to 457 mL/h in the group that received 75 mg of @DRUG$ per day).",DDI-false
18777,DDI-MedLine.d99.s11.p5,"The clearance by means of 4-hydroxytolbutamide and carboxytolbutamide was significantly reduced in both groups (ie, from 901 mL/h to 318 mL/h in the group that received 150 mg of @DRUG$ per day and from 723 mL/h to 457 mL/h in the group that received 75 mg of @DRUG$ per day).",DDI-false


## FASE PRELIMINARE MODELLO

In [4]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

In [5]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train.label.to_numpy().reshape(-1, 1)
)

five_hot_dev = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  dev.label.to_numpy().reshape(-1, 1)
)

In [57]:
print(np.unique(train.label))
print(categories)

['DDI-advise' 'DDI-effect' 'DDI-false' 'DDI-int' 'DDI-mechanism']
[['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise', 'DDI-int']]


In [60]:
class_weight = compute_class_weight("balanced", classes=np.unique(train.label), y=train.label.to_numpy()) 
class_weight = {0: np.float32(class_weight[np.where(np.unique(train.label) == categories[0][0])[0][0]]),
                1: np.float32(class_weight[np.where(np.unique(train.label) == categories[0][1])[0][0]]),
                2: np.float32(class_weight[np.where(np.unique(train.label) == categories[0][2])[0][0]]),
                3: np.float32(class_weight[np.where(np.unique(train.label) == categories[0][3])[0][0]]),
                4: np.float32(class_weight[np.where(np.unique(train.label) == categories[0][4])[0][0]])}
class_weight

{0: 0.23707865, 1: 3.9701903, 2: 3.0988448, 3: 5.9333334, 4: 25.724657}

In [7]:
my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

sentences_train = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]
sentences_dev = [my_text_to_word_sequence(sentence) for sentence in dev['sentence']]

In [8]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)

for i, sentence in enumerate(sentences_dev):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)

print(f'Il massimo è {max}')

Il massimo è 92


In [9]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

embedded_devset = np.zeros(shape=(len(sentences_dev), max, 300))
for i, sentence in enumerate(sentences_dev):
  for j, word in enumerate(sentence):
    try:
      embedded_devset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

## CREAZIONE MODELLO

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.LSTM(64, recurrent_dropout=0.2))
model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 93,765
Trainable params: 93,765
Non-trainable params: 0
_________________________________________________________________


In [ ]:
result = model.fit(embedded_trainset, five_hot_train, epochs=2, batch_size=25)

Epoch 1/2
752/752 [==============================] - 94s 122ms/step - loss: 0.7067 - accuracy: 0.8377
Epoch 2/2
752/752 [==============================] - 92s 123ms/step - loss: 0.6300 - accuracy: 0.8429


In [ ]:
frase = sentences_test[25:50]
boh = np.zeros(shape=(len(frase), max, 300))
for i, sentence in enumerate(frase):
  for j, word in enumerate(sentence):
    try:
      boh[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

res = model.predict(boh)
[row[2] for row in res]

[0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.8470076,
 0.84700775,
 0.84700775,
 0.84700775,
 0.84700775,
 0.84700763,
 0.84700763,
 0.84700775,
 0.84700763,
 0.84700763,
 0.8470076,
 0.84700763,
 0.84700763,
 0.84700763,
 0.84700763,
 0.8470076,
 0.8470076]

**Si osserva che tutti i sample hanno la stessa probabilità di rientrare nella terza classe (che corrisponde a "DDI-false"). Si sottolinea che ~0.84 è la percentuale di DDI-false nel training set.**

## MODELLO CON SAMPLE PIÙ EQUILIBRATI (MENO DDI-FALSE)

In [ ]:
reduced_train = pd.concat([train.loc[train['label'] == 'DDI-false'].sample(n=3000), train.loc[train['label'] != 'DDI-false']]).sample(frac=1)

In [ ]:
five_hot_train_reduced = OneHotEncoder(sparse=False).fit_transform(
  reduced_train.label.to_numpy().reshape(-1, 1)
)

In [ ]:
reduced_sentences_train = [my_text_to_word_sequence(sentence) for sentence in reduced_train['sentence']]

In [ ]:
reduced_embedded_trainset = np.zeros(shape=(len(reduced_sentences_train), max, 300))
for i, sentence in enumerate(reduced_sentences_train):
  for j, word in enumerate(sentence):
    try:
      reduced_embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
model_reduced = keras.Sequential()
model_reduced.add(keras.layers.Input(shape=(max, 300)))
model_reduced.add(keras.layers.LSTM(64, recurrent_dropout=0.2))
model_reduced.add(keras.layers.Dense(5, activation='softmax'))
model_reduced.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.004), metrics=['accuracy'])
model_reduced.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 93,765
Trainable params: 93,765
Non-trainable params: 0
_________________________________________________________________


In [ ]:
result = model_reduced.fit(reduced_embedded_trainset, five_hot_train_reduced, epochs=2, batch_size=25)

Epoch 1/2
238/238 [==============================] - 35s 138ms/step - loss: 1.3455 - accuracy: 0.4911
Epoch 2/2
238/238 [==============================] - 33s 137ms/step - loss: 1.2386 - accuracy: 0.5110


## OPTUNA

In [ ]:
%pip install optuna

In [10]:
import optuna
from sklearn.metrics import confusion_matrix

In [11]:
def metrics_2(t_labels, t_predictions):
  numeric_labels = np.argmax(t_labels, axis=1)
  numeric_predictions = np.argmax(t_predictions, axis=1)
  matrix = confusion_matrix(numeric_labels, numeric_predictions)
  print(matrix)
  FP = (matrix.sum(axis=0) - np.diag(matrix))[1:]
  FN = (matrix.sum(axis=1) - np.diag(matrix))[1:]
  TP = (np.diag(matrix))[1:]
  overall_fp = np.sum(FP)
  overall_fn = np.sum(FN)
  overall_tp = np.sum(TP)
  overall_precision = overall_tp / (overall_tp + overall_fp)
  overall_recall = overall_tp / (overall_tp + overall_fn)
  overall_f_score = 2 * overall_precision * overall_recall / (overall_precision + overall_recall)
  return overall_f_score

In [14]:
def objective(trial):
    units = trial.suggest_int('units', 40, 140)
    recurrent_dropout = trial.suggest_float('dropout', 0.2, 0.8, step=0.01)
    
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(max, 300)))
    model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=units,
                                                                 recurrent_dropout=recurrent_dropout,
                                                                 activation='tanh')))
    
    model.add(keras.layers.Dense(5, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    batch_size = trial.suggest_int('batch_size', 50, 128)
    result = model.fit(embedded_trainset,
                       five_hot_train,
                       epochs=2,
                       batch_size=batch_size,
                       class_weight=class_weight,
                       callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                                patience=10)])

    return metrics_2(model.predict(embedded_devset), five_hot_dev)

In [ ]:
study = optuna.create_study(direction='maximize',storage="sqlite:///models.db", study_name="DDI")
study.optimize(objective, n_trials=300, n_jobs=50)